In [109]:
import requests
import json
from dateutil.parser import parse

In [145]:
nickname = '포뇨의브루스'
apikey = 'X1o3cmgN7nsBDsInu4km3XtlPD051oPC'
wordType = 'match'
main_url = 'https://api.neople.co.kr/cy/'
player_url = 'https://api.neople.co.kr/cy/players/'
api = main_url+'players?nickname='+nickname+'&wordType='+wordType+'&apikey='+apikey
playerdata = requests.get(api)
playerId = playerdata.json()['rows'][0]['playerId']
gameTypeId = 'normal'
#검색 시작일
startDate='2022-01-01'
endDate='2022-03-30'
limit = 100
next1 = 'next'

In [146]:
api2 = player_url+playerId+'?apikey='+apikey

In [147]:
api2

'https://api.neople.co.kr/cy/players/776727c2572eaae42249361d2c925941?apikey=X1o3cmgN7nsBDsInu4km3XtlPD051oPC'

In [148]:
playerinfo = requests.get(api2)

In [149]:
playerinfo.json()

{'playerId': '776727c2572eaae42249361d2c925941',
 'nickname': '포뇨의브루스',
 'grade': 90,
 'clanName': '힣',
 'ratingPoint': None,
 'maxRatingPoint': None,
 'tierName': None,
 'records': [{'gameTypeId': 'rating',
   'winCount': 0,
   'loseCount': 0,
   'stopCount': 0},
  {'gameTypeId': 'normal', 'winCount': 6, 'loseCount': 8, 'stopCount': 0}]}

In [150]:
cc = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate

In [151]:
playdata = requests.get(cc)

In [152]:
playdata.json()

{'playerId': '776727c2572eaae42249361d2c925941',
 'nickname': '포뇨의브루스',
 'grade': 90,
 'clanName': '힣',
 'ratingPoint': None,
 'maxRatingPoint': None,
 'tierName': None,
 'records': [{'gameTypeId': 'rating',
   'winCount': 0,
   'loseCount': 0,
   'stopCount': 0},
  {'gameTypeId': 'normal', 'winCount': 6, 'loseCount': 8, 'stopCount': 0}],
 'matches': {'date': {'start': '2022-01-01 00:00', 'end': '2022-03-30 00:00'},
  'gameTypeId': 'normal',
  'next': 'a99999d9b3d32fe0f3dd9a4c278119a12839d31b524ab80a056a077a3dc547dea4a7125effe9e729a96bfb166b19fac0c18a75ff2ad7d6810a4643d51b81af59ec861250d7677913e21074fb4e88f460',
  'rows': [{'date': '2022-02-21 00:21',
    'matchId': '90be716790554803fa088ac94aa9423257317a9dee49d416bbdf80494376ec7d',
    'map': {'mapId': '103', 'name': '브리스톨'},
    'playInfo': {'result': 'lose',
     'random': False,
     'partyUserCount': 5,
     'partyInfo': [{'playerId': '6ccf7c1cf45c570417402b02eb8b8ccd',
       'nickname': '로리와우리들의',
       'characterId': '163cf60f

In [141]:
cc

'https://api.neople.co.kr/cy/players/776727c2572eaae42249361d2c925941/matches?gameTypeId=normal&apikey=X1o3cmgN7nsBDsInu4km3XtlPD051oPC&startDate=2018-09-01'